# **Setup**

* https://docs.ultralytics.com/models/yolo11

In [2]:
import os
import cv2
import datetime
import time
from ultralytics import YOLO

# !pip install azure-storage-blob --quiet
from azure.storage.blob import BlobServiceClient

# **Get Camera Details**

In [1]:
import cv2

def list_cameras():
    """
    Detect available cameras and return a list of their indices and properties.
    """
    camera_index = 0
    available_cameras = []

    while True:
        cap = cv2.VideoCapture(camera_index)
        if not cap.isOpened():
            break

        is_reading, _ = cap.read()
        if is_reading:
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            available_cameras.append({
                'index': camera_index,
                'resolution': f"{width}x{height}",
                'functional': True
            })
        else:
            available_cameras.append({
                'index': camera_index,
                'functional': False
            })

        camera_index += 1

    return available_cameras

In [3]:
# Get the list of available cameras
camera_list = list_cameras()

# Print camera information
for camera in camera_list:
    print(f"Camera {camera['index']} - Resolution: {camera.get('resolution', 'N/A')}, Functional: {camera['functional']}")

# You can now choose a camera index to use with cv2.VideoCapture(index)

Camera 0 - Resolution: 640x480, Functional: True
Camera 1 - Resolution: 1280x720, Functional: True
Camera 2 - Resolution: 1024x768, Functional: True


# **Camera Details**

In [2]:
# Camera details
CAMERA_ID = "CAM1"
CAMERA_NAME = "Camera 1"
LOCATION_NAME = "Main Entrance"
LOCATION_COORDINATES = "37.7749° N, 122.4194° W"  # Example coordinates
MAX_DURATION = 4 * 60 * 60  # 4 hours in seconds

# Azure Blob Storage details
AZURE_CONNECTION_STRING = "your_azure_storage_connection_string"
CONTAINER_NAME = "your_container_name"

In [ ]:
# Function For Capturing Video

def capture_video(camera_id):
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print(f"Error: Unable to access camera {camera_id}")
        return None

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{CAMERA_NAME}_{timestamp}.avi"
    out = cv2.VideoWriter(filename, fourcc, 20.0, (640, 480))

    start_time = time.time()
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read from camera")
            break

        out.write(frame)
        cv2.imshow(CAMERA_NAME, frame)

        if time.time() - start_time > MAX_DURATION:
            break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    return filename

In [12]:
# Create a VideoCapture object
cap = cv2.VideoCapture(1)

# Check if camera opened successfully
if not cap.isOpened():
    print("Unable to open camera")

# Loop until the user hits the 'q' key on their keyboard
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        # Display the resulting frame
        cv2.imshow('Frame', frame)

        # Quit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# When everything done, release the VideoCapture and destroy windows
cap.release()
cv2.destroyAllWindows()

# **Object Detection Model**

In [9]:
# Use YOLOv11 for object detection
# Load YOLOv11 model
from ultralytics import YOLO
model = YOLO('yolov11n.pt')  # Load the YOLO11 model

# Perform object detection on the captured frame
results = model(frame)  # Assuming 'frame' is the captured image

# Display results
for result in results:
    boxes = result.boxes.xyxy  # Get bounding boxes
    for box in boxes:
        x1, y1, x2, y2 = map(int, box[:4])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(frame, f"ID: {int(box[5])}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Draw ID
        cv2.putText(frame, f"Class: {int(box[4])}", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.putText(frame, f"Confidence: {box[6]:.2f}", (x1, y1 - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    cv2.imshow('YOLOv11 Detection', frame)  # Display the frame with detections
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

FileNotFoundError: [Errno 2] No such file or directory: 'yolov11n.pt'

# **Data Upload**

In [ ]:
# Upload video to Azure Blob Storage
def upload_to_azure():
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
    blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=new_video_filename)
    
    with open(new_video_filename, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    
    print("Video uploaded to Azure Blob Storage successfully.")

upload_to_azure()

# **Live Detection**

In [11]:
import cv2
import datetime
import csv
import os
from ultralytics import YOLO

def detect_objects_live(camera_idx=0,camera_id="CAM1", camera_name="Camera 1", location_name="Main Entrance", 
                         location_coordinates="37.7749° N, 122.4194° W", 
                         yolo_model_path="yolov11n.pt", target_labels=None, log_file="detection_log.csv"):
    """
    Captures live video, detects objects using YOLOv8, and logs detected objects to a CSV file.

    Parameters:
    - camera_id (str): Unique ID for the camera
    - camera_name (str): Name of the camera
    - location_name (str): Location description
    - location_coordinates (str): Coordinates of the location
    - yolo_model_path (str): Path to the YOLOv8 model
    - target_labels (list): List of labels to detect (e.g., ["person", "car"])
    - log_file (str): Name of the CSV log file
    """

    # Load YOLOv8 model
    model = YOLO(yolo_model_path)

    # Open camera
    cap = cv2.VideoCapture(camera_idx)  # Change index if using multiple cameras

    # Ensure log file exists and has a header
    if not os.path.exists(log_file):
        with open(log_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Time", "Camera ID", "Object", "Confidence"])

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Object detection using YOLOv8
        results = model(frame)

        detected_objects = []

        # Draw bounding boxes and log detected objects
        for result in results:
            for i, box in enumerate(result.boxes.xyxy):
                x1, y1, x2, y2 = map(int, box[:4])
                confidence = result.boxes.conf[i].item()
                label_index = int(result.boxes.cls[i].item())
                label = result.names[label_index]

                # Only process if label is in target_labels (or if target_labels is None, detect everything)
                if target_labels is None or label in target_labels:
                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"{label} {confidence:.2f}", (x1, y1 - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                    # Log detected object
                    detected_objects.append([datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), camera_id, label, round(confidence, 2)])

        # Save detected objects to CSV log
        if detected_objects:
            with open(log_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(detected_objects)

        # Overlay metadata
        cv2.putText(frame, f"{camera_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"{location_name}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"{location_coordinates}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Live Object Detection', frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

    print(f"Detections logged in: {log_file}")

In [13]:
# Example function call
detect_objects_live(camera_idx=1,yolo_model_path='yolo11s.pt') 


0: 384x640 (no detections), 130.5ms
Speed: 1.5ms preprocess, 130.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 124.9ms
Speed: 1.7ms preprocess, 124.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 139.1ms
Speed: 1.4ms preprocess, 139.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 120.7ms
Speed: 1.4ms preprocess, 120.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 110.6ms
Speed: 1.3ms preprocess, 110.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 120.2ms
Speed: 2.0ms preprocess, 120.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 119.2ms
Speed: 1.5ms preprocess, 119.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 116.8ms
Speed: 1.9ms prepr